Riding the Wave

Convection Problems

This module will explore the convection equation in more depth!



Conservation Laws

Conservation of Mass:

Rate of change of mass in the CV + rate of cahnge of mass across CS = 0

COM:

$$\frac{\partial}{\partial t} \int_{cv} \rho dV + \int_{cs} \rho u dA =0$$

Since CV is small we can take $\rho$ as uniform quantity inside it. 

$$\frac{\partial}{\partial t} \int_{cv} \rho dV \rightarrow \frac{\partial \rho}{\partial t}dAdx$$

Total flux of mass across the CS

$$\int_{cs} pudA \rightarrow \frac{\partial}{\partial x}(pu)dAdx$$

$$\frac{\partial \rho}{\partial t} + \frac{\partial}{\partial x}(pu) = 0$$

In [1]:
from IPython.display import YouTubeVideo
YouTubeVideo('35unQgSaT88')


General Conservation Laws

A general conservation law for quantity e is:

$$\frac{\partial}{\partial t} \int_{cv} edV + \oint_{cs} \vec{F} \cdot d\vec{A} = 0$$

Apply theorem of Gauss to the second integral

Recall nonlinear convection was:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x}= 0$$

We can rewrite this as:

$$\frac{\partial u}{\partial t} + \frac{\partial }{\partial x} \left( \frac{u^2}{2} \right ) = 0$$

which is the conservative form of the nonlinear convection equation:

$$F = \frac{u^2}{2}$$





Traffic Flow Model

Consider the linear function:

$$u(\rho) = u_{max} \left(1-\frac{\rho}{\rho_{max}}\right)$$

Experimental Observations:

1) All drivers will approach a max velocity $u_max$ when the road is empty
2) If the road is completely jampacked ($\rho \rightarrow \rho_{max}$$), velocity goes to zero

apply conservation law to traffic density $\rho$

$$\frac{\partial \rho}{\partial t} + \frac{\partial F}{\partial x} = 0$$

Where F is traffic flux

$$F = \rho u_{max} \left(1-\frac{\rho}{\rho_{max}}\right)$$

Green Light!

Lets say we are examining a road of length 4 where speed limit is $u_{max} = 1$, fitting 10 cars per unit length ($\rho_{max} =10$). We have intersection with a red light at x=2. At light traffic is bumper to bumper and density decreases linearly to zero as we approach beginning of our road, ahead of light road is clear. Mathematically represent this:

\begin{equation}\rho(x,0) = \left\{ \begin{array}{cc}
\rho_{\rm max}\frac{x}{2} & 0 \leq x < 2  \\
0 & 2 \leq x \leq 4 \\ \end{array} \right.\end{equation}



In [2]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from matplotlib import rcParams
rcParams['font.family']='serif'
rcParams['font.size']=16


In [ ]:
def rho_green_light(nx, rho_light):
    
    """Computes green light inital condition with shock, and linear distribution behind"""
    
    
    rho = numpy.arrange(nx)*2./nx*rho_light #before stoplight
    rho
    